In [2]:
import os
import string

# Base directory
base_dir = "writing_motion_dataset"

# Subfolder categories
digits = [str(i) for i in range(10)]  # Digits 0-9
uppercase = list(string.ascii_uppercase)  # Uppercase letters A-Z
lowercase = list(string.ascii_lowercase)  # Lowercase letters a-z

# Create the base directory if it doesn't exist
if not os.path.exists(base_dir):
    os.makedirs(base_dir)
    print(f"Created base directory: {base_dir}")

# Create subfolders for digits, uppercase, and lowercase letters
for label in digits + uppercase + lowercase:
    folder_path = os.path.join(base_dir, label)
    os.makedirs(folder_path, exist_ok=True)
    print(f"Created folder: {folder_path}")

print("Directory structure created successfully!")

Created base directory: writing_motion_dataset
Created folder: writing_motion_dataset\0
Created folder: writing_motion_dataset\1
Created folder: writing_motion_dataset\2
Created folder: writing_motion_dataset\3
Created folder: writing_motion_dataset\4
Created folder: writing_motion_dataset\5
Created folder: writing_motion_dataset\6
Created folder: writing_motion_dataset\7
Created folder: writing_motion_dataset\8
Created folder: writing_motion_dataset\9
Created folder: writing_motion_dataset\A
Created folder: writing_motion_dataset\B
Created folder: writing_motion_dataset\C
Created folder: writing_motion_dataset\D
Created folder: writing_motion_dataset\E
Created folder: writing_motion_dataset\F
Created folder: writing_motion_dataset\G
Created folder: writing_motion_dataset\H
Created folder: writing_motion_dataset\I
Created folder: writing_motion_dataset\J
Created folder: writing_motion_dataset\K
Created folder: writing_motion_dataset\L
Created folder: writing_motion_dataset\M
Created fo

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
from collections import deque

# Mediapipe setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

# Directory to save dataset images
base_dir = "writing_motion_dataset"

# Check if the base directory exists
if not os.path.exists(base_dir):
    print(f"Error: The directory '{base_dir}' does not exist.")
    exit()

# Smoothing fingertip position
def smooth_coordinates(coord_queue, new_coord, max_len=5):
    coord_queue.append(new_coord)
    if len(coord_queue) > max_len:
        coord_queue.popleft()
    avg_x = int(np.mean([x for x, y in coord_queue]))
    avg_y = int(np.mean([y for x, y in coord_queue]))
    return avg_x, avg_y

# Function to record a single sample for a given label
def record_sample(label):
    label_dir = os.path.join(base_dir, label)

    # Check if the label folder exists
    if not os.path.exists(label_dir):
        print(f"Error: The label '{label}' does not exist in the dataset directory.")
        return

    # Initialize webcam
    cap = cv2.VideoCapture(0)

    # Gesture collection variables
    drawing = False
    canvas = None  # Initialize blank canvas for motion
    fingertip_coords = deque(maxlen=5)
    trajectory = []  # Store fingertip trajectory
    prev_coords = None
    count = len(os.listdir(label_dir)) + 1  # Determine the next sample number

    print(f"Starting gesture collection for label '{label}'. Press 'd' to toggle drawing, 's' to save, 'q' to quit.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Unable to access the camera.")
            break

        # Flip for mirror effect and convert to RGB
        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Initialize canvas on the first frame
        if canvas is None:
            canvas = np.zeros_like(frame)

        # Process hand landmarks
        results = hands.process(rgb_frame)

        if results.multi_hand_landmarks:
            # Get index finger tip
            hand_landmarks = results.multi_hand_landmarks[0]
            fingertip = hand_landmarks.landmark[8]
            h, w, _ = frame.shape
            x, y = int(fingertip.x * w), int(fingertip.y * h)

            # Smooth fingertip motion
            x, y = smooth_coordinates(fingertip_coords, (x, y))

            if drawing:
                trajectory.append((x, y))  # Append coordinates to trajectory
                if prev_coords is not None:
                    # Draw motion trail
                    cv2.line(canvas, prev_coords, (x, y), (255, 255, 255), thickness=5)
                prev_coords = (x, y)
            else:
                prev_coords = None  # Reset when not drawing

            # Draw hand landmarks
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Display canvas and instructions
        combined = cv2.addWeighted(frame, 0.5, canvas, 0.5, 0)
        cv2.putText(combined, f"Label: {label}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(combined, f"Samples: {count}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(combined, "Press 'd' to start/stop drawing", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(combined, "Press 's' to save gesture", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(combined, "Press 'q' to quit", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        cv2.imshow("Draw Gestures", combined)

        # Handle keypress
        key = cv2.waitKey(1) & 0xFF
        if key == ord('d'):  # Toggle drawing
            drawing = not drawing
            print("Drawing mode:", "ON" if drawing else "OFF")
        elif key == ord('c'):  # Clear canvas
            canvas = np.zeros_like(frame)
            trajectory = []
        elif key == ord('q'):  # Quit
            break
        elif key == ord('s'):  # Save gesture
            if not drawing and canvas is not None and len(trajectory) > 1:
                # Save trajectory as a heatmap
                motion_image = np.zeros((480, 640), dtype=np.uint8)
                for (x, y) in trajectory:
                    cv2.circle(motion_image, (x, y), 3, 255, -1)

                # Save heatmap to label folder
                filename = os.path.join(label_dir, f"sample_{count}.jpg")
                cv2.imwrite(filename, motion_image)
                count += 1
                print(f"Saved sample {count} for label '{label}'")

                # Reset canvas and trajectory
                canvas = np.zeros_like(frame)
                trajectory = []

    cap.release()
    cv2.destroyAllWindows()

# Main loop
while True:
    # Get user input for label
    label = input("Enter the label (0-9, A-Z, a-z) for gesture collection (or 'q' to quit): ")

    # Quit if user inputs '@'
    if label.lower() == '@':
        print("Exiting the program.")
        break

    # Record sample for the given label
    record_sample(label)

KeyboardInterrupt: Interrupted by user